In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img , img_to_array
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import relu, elu
from tensorflow.keras.layers import Dropout, Conv2D, Dense, MaxPool2D, Flatten

In [ ]:
train_csv = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test_csv = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
sub_csv = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")

In [ ]:
def image_gen_norm(image_path):
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (128, 128))
    return image
train = []
train_label = train_csv["Pawpularity"]
for i in range(train_csv.shape[0]):
    img = image_gen_norm("../input/petfinder-pawpularity-score/train/"+train_csv["Id"][i]+".jpg")
    train.append(img)
    
test = []
for i in range(test_csv.shape[0]):
    img = image_gen_norm("../input/petfinder-pawpularity-score/test/"+test_csv["Id"][i]+".jpg")
    test.append(img)
        

In [ ]:
train_arr = np.array(train)

In [ ]:
adam = Adam(0.001)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=4, kernel_size=(3,3),strides=2, activation="relu", padding="same", input_shape=(128,128,3)))
# model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu"))
# model.add(MaxPooling2D())
model.add(Conv2D(filters=8, kernel_size=(3,3),strides=2, activation="relu", padding="same"))
# model.add(Conv2D(filters=16, kernel_size=(3,3), activation="relu"))
# model.add(MaxPooling2D())
model.add(Conv2D(filters=16, kernel_size=(3,3),strides=2, activation="relu", padding="same"))
# model.add(Conv2D(filters=8, kernel_size=(3,3), activation="relu"))
model.add(Conv2D(filters=32, kernel_size=(3,3),strides=2, activation="relu", padding="same"))

model.add(Flatten())
model.add(Dense(units = 128, activation="relu"))
# model.add(Dropout(0.3))
# model.add(Dense(units = 64, activation="relu"))
model.add(Dropout(0.5))
# model.add(Dense(units = 32, activation="relu"))
# model.add(Dropout(0.1))
model.add(Dense(units = 1, activation="relu"))

In [ ]:
model.compile(loss="mse", optimizer = "Adam", metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])
predictor = model.fit(train_arr,train_label,validation_split=0.2,epochs=25,batch_size=32)

In [ ]:
xtest = np.array(test)

In [ ]:
pred = model.predict(xtest)

final=pd.DataFrame()
final['Id']=test_csv['Id']
final['Pawpularity']=pred
final.to_csv('submission.csv',index=False)